# Import the necessary Libraries

In [1]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from PIL import Image
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Defining some constants

In [2]:
diseaseType = ['Dementia dataset', 'Breast Cancer dataset', 'Pneumonia dataset', 'Retinal Disease dataset', 'Skin Disease dataset']
class_names = [
    ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented'],
    ['Benign', 'Malignant', 'Normal'],
    ['NORMAL', 'PNEUMONIA'],
    ['ARMD', 'DN', 'DR', 'MH', 'NORMAL', 'ODC'],
    ['Actinic keratosis', 'Atopic Dermatitis', 'Benign Keratosis', 'Dermatofibroma', 'Melanocytic nevus', 'Melanoma', 'Squamous cell carcinoma', 'Tinea Ringworm', 'Vascular lesion']
]

In [3]:
# resolution of images on which the models are trained
resolution=[
    (128, 128),
    (128, 128),
    (256, 256),
    (224, 224),
    (224, 224)
]

# Directories of the  random images to be tested

In [9]:
img_dirs=[
          'benign.jpg',
          'mild.jpg',
          'AK.jpg',
          'MH.jpg',
          'pneu.jpg',
          'normallung.jpg',
          'Dma.jpg',
          'normal.jpg',
          'N1.jpg',
          'ARMD1.jpg'
        ]
img_labels=['Benign Breast Cancer', 'Mild Dementia', 'Skin Disease: Actinic keratosis','Retinal Disease: MH', 'Pneumonic Lungs', 'Normal Lung', 'Skin disease DMA', 'No Problem in eye', 'No Problem in eye', 'Retinal Disease: ARMD']

# Loading the models

In [5]:
diseaseTypeModel=joblib.load('model.joblib')
pcamodel=joblib.load('pca_model.joblib')
diseaseTypeModel.classes_

c:\Users\Sounak\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Sounak\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([0, 1, 2, 3, 4])

In [6]:
diseaseModels = [
    tf.keras.models.load_model('BrainMRIcnn'),
    tf.keras.models.load_model('BreastCancerModel'),
    joblib.load('pneumoniaSVM.pkl'),
    tf.keras.models.load_model('RetinalDiseaseCNN'),
    tf.keras.models.load_model('SkinDiseaseModel')
]

# Predicting the Diseases

In [7]:
def findDiseaseType(fname):
    img = Image.open(fname)

    g_img = img.convert('L')    
    g_img = g_img.resize((64, 64)) # 64 is the size we trained the model on
    img_array = np.array(g_img).flatten()

    img_array=np.expand_dims(img_array, axis=0)

    img_pca=pcamodel.transform(img_array)
    predicted_label=diseaseTypeModel.predict(img_pca)

    return predicted_label[0]

In [12]:
for filename, original_label in zip(img_dirs, img_labels):
    predicted_label=findDiseaseType(filename)
    print ("The type of disease detected is:", diseaseType[predicted_label])

    _img = image.load_img(filename, target_size=resolution[predicted_label])

    if predicted_label==2: # flattenning the image since the Dimentia is trained on ML model and not on CNN model
        _img = np.array(_img)
        _img = _img.flatten()

    _img_array = np.expand_dims(_img, axis=0)
    pld=diseaseModels[predicted_label].predict(_img_array)
    label = np.argmax(pld)

    print("Predicted label is : ", class_names[predicted_label][label])
    print("Original label is : ", original_label,'\n\n')

The type of disease detected is: Breast Cancer dataset
1/1 [==============================] - 0s 48ms/step
Predicted label is :  Benign
Original label is :  Benign Breast Cancer 


The type of disease detected is: Dementia dataset
1/1 [==============================] - 0s 20ms/step
Predicted label is :  Non Demented
Original label is :  Mild Dementia 


The type of disease detected is: Skin Disease dataset
1/1 [==============================] - 0s 113ms/step
Predicted label is :  Actinic keratosis
Original label is :  Skin Disease AK 


The type of disease detected is: Retinal Disease dataset
1/1 [==============================] - 0s 99ms/step
Predicted label is :  MH
Original label is :  Retinal Disease: MH 


The type of disease detected is: Pneumonia dataset
Predicted label is :  NORMAL
Original label is :  Pneumonic Lungs 


The type of disease detected is: Pneumonia dataset
Predicted label is :  NORMAL
Original label is :  Normal Lung 


The type of disease detected is: Skin Disea